In [28]:
import os
from requests import Session
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from random import randint

# Маскируемся под браузер.
session = Session()
session.headers.update({ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36' })
get = session.get

In [29]:
RUSFENCING_BASE_URL = 'http://rusfencing.ru'

def get_page(url):
    res = get(url)
    return BeautifulSoup(res.text, 'html.parser')

In [30]:
def get_fencers_page(num = 1):
    url = '{}/sportsmen.php?PAGEN_1={}'.format(RUSFENCING_BASE_URL, num)
    return get_page(url)

def get_fencers_page_count():
    page = get_fencers_page()
    end = page.find('a', string='Конец')['href']
    r = re.search(r'PAGEN_1=(\d+)', end)
    if r and len(r.groups()) == 1:
        count = r.group(1)
        return int(count)
    else:
        return None

In [31]:
def get_rusfencing_id(cell):
    a = cell.find('a')['href']
    r = re.search(r'ID=(\d+)', a)
    if r and len(r.groups()) == 1:
        id = r.group(1)
        return int(id)
    else:
        return None

def normalize_degree(degree):
    # NOTE: Может, как-то в одну строку переписать?
    if not degree or degree == 'б/р':
        return None
    return degree

def get_all_fencers_on_page(num = 1):
    page = get_fencers_page(num)
    fencers = []
    rows = page.findAll('tr')
    for row in rows:
        cells = row.findAll('td')
        if len(cells) == 10:
            fencer = {
                'rusfencing_id': get_rusfencing_id(cells[1]),
                'license_number': cells[0].text,
                'name': cells[1].text,
                'sex': cells[2].text,
                'birthdate': cells[3].text,
                'weapon': cells[4].text,
                'degree': normalize_degree(cells[5].text),
                'subject': cells[6].text
            }
            fencers.append(fencer)
    return fencers

In [32]:
def get_all_fencers(filename = 'fencers.csv'):
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        fencers_page_count = get_fencers_page_count()
        fencers = []
        for i in range(1, fencers_page_count + 1):
            sleep(randint(8,15))
            fencers.extend(get_all_fencers_on_page(i))
        df = pd.DataFrame(fencers)
        df.to_csv('fencers.csv')
    return df

In [35]:
get_all_fencers().sample(5)

,Unnamed: 0,birthdate,degree,license_number,name,rusfencing_id,sex,subject,weapon
7038,7038,27.08.2001,КМС,27082001002,Моисеева Екатерина Дмитриевна,188843,Ж,ТАТ,шпага
6157,6157,03.06.2004,NaN,3062004007,Ломтева Арина Александровна,935809,Ж,МОС,сабля
10461,10461,18.04.2002,II разряд,18042002005,Тиунова Виктория Игоревна,757158,Ж,КИР,шпага
982,982,11.02.2003,II разряд,11022003004,Беляев Кирилл Владимирович,946499,М,МОС,шпага
9262,9262,03.04.2010,NaN,3042010000,Сафин Айдар Дамирович,1471262,М,УЛЬ,сабля
